In [1]:
using WGLMakie
using GLMakie
using Graphs
using JSServe
using NetworkLayout
using GraphMakie
using CSV
using DataFrames
using LinearAlgebra

In [2]:
flows = DataFrame(CSV.File("whole_cell_flow_data.csv"))

Row,Column1,data
,String,Float64
1,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse),3851.0
2,1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse),16095.0
3,1PFRUCTPHOSN-RXN,1.23812e6
4,1TRANSKETO-RXN (reverse),7974.0
5,2-DEHYDROPANTOATE-REDUCT-RXN (reverse),1131.0
6,2-ISOPROPYLMALATESYN-RXN,56297.0
7,2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN,300.0
8,2-OCTAPRENYL-6-OHPHENOL-METHY-RXN,300.0
9,2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN,300.0


In [3]:
stoichiometry = DataFrame(CSV.File("whole_cell_s_matrix (1).csv"))

Row,Column1,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse),1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse),1PFRUCTPHOSN-RXN,1TRANSKETO-RXN (reverse),2-DEHYDROPANTOATE-REDUCT-RXN (reverse),2-ISOPROPYLMALATESYN-RXN,2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN,2-OCTAPRENYL-6-OHPHENOL-METHY-RXN,2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN,2-OCTAPRENYLPHENOL-HYDROX-RXN,2.3.1.157-RXN,2.5.1.19-RXN,2.5.1.64-RXN,2.7.1.148-RXN,2.7.7.60-RXN,2.8.1.6-RXN,2OXOGLUTARATEDEH-RXN,2PGADEHYDRAT-RXN,2TRANSKETO-RXN (reverse),3-CH3-2-OXOBUTANOATE-OH-CH3-XFER-RXN-CPD-12996/2-KETO-ISOVALERATE/WATER//2-DEHYDROPANTOATE/CPD-1301.63.,3-DEHYDROQUINATE-DEHYDRATASE-RXN,3-DEHYDROQUINATE-SYNTHASE-RXN,3-OCTAPRENYL-4-OHBENZOATE-DECARBOX-RXN,3.2.2.10-RXN-CMP/WATER//RIBOSE-5P/CYTOSINE.30.,3.2.2.10-RXN-UMP/WATER//RIBOSE-5P/URACIL.28.,325-BISPHOSPHATE-NUCLEOTIDASE-RXN,3PGAREARR-RXN (reverse),4OHBENZOATE-OCTAPRENYLTRANSFER-RXN,5.1.3.20-RXN,5.4.2.10-RXN-CPD-13469//GLUCOSAMINE-1P.26.,6PGLUCONOLACT-RXN,ACECOATRANS-RXN-PROPIONYL-COA/ACET//PROPIONATE/ACETYL-COA.42.,ACETATEKIN-RXN,ACETOLACTREDUCTOISOM-RXN (reverse),ACETOLACTSYN-RXN,ACETOOHBUTREDUCTOISOM-RXN (reverse),ACETOOHBUTSYN-RXN,ACETYL-COA-ACETYLTRANSFER-RXN,ACETYLGLUTKIN-RXN,ACETYLORNDEACET-RXN,ACETYLORNTRANSAM-RXN (reverse),ACONITATEDEHYDR-RXN,ACSERLY-RXN,ADCLY-RXN,ADENOSYLHOMOCYSTEINE-NUCLEOSIDASE-RXN,ADENPRIBOSYLTRAN-RXN (reverse),ADENYL-KIN-RXN,ADENYLOSUCCINATE-SYNTHASE-RXN,ADENYLYLSULFKIN-RXN,ADOMET-DMK-METHYLTRANSFER-RXN,AICARSYN-RXN,AICARTRANSFORM-RXN-10-FORMYL-THF/AICAR//THF/PHOSPHORIBOSYL-FORMAMIDO-CARBOXAMIDE.62.,AIRS-RXN,ALANINE-AMINOTRANSFERASE-RXN (reverse),ALARACECAT-RXN,AMPSYN-RXN,ANTHRANSYN-RXN,ARGSUCCINLYA-RXN,ARGSUCCINSYN-RXN,ASNSYNA-RXN,ASPAMINOTRANS-RXN,ASPARTASE-RXN (reverse),ASPARTATE-SEMIALDEHYDE-DEHYDROGENASE-RXN (reverse),ASPARTATEKIN-RXN,ASPCARBTRANS-RXN,ASPDECARBOX-RXN,ATPPHOSPHORIBOSYLTRANS-RXN (reverse),ATPSYN-RXN (reverse),BRANCHED-CHAINAMINOTRANSFERILEU-RXN (reverse),BRANCHED-CHAINAMINOTRANSFERLEU-RXN (reverse),BRANCHED-CHAINAMINOTRANSFERVAL-RXN (reverse),CARDIOLIPSYN-RXN-CPD-8260//CPD-12824/GLYCEROL.29.,CDPDIGLYSYN-RXN-CTP/CPD0-1422/PROTON//CPD-12815/PPI.36.,CDPKIN-RXN,CHORISMATE-SYNTHASE-RXN,CHORPYRLY-RXN,CITSYN-RXN,CPM-KDOSYNTH-RXN,CYSTATHIONINE-BETA-LYASE-RXN,D-PPENTOMUT-RXN (reverse),DAPASYN-RXN,DEOXYADENPHOSPHOR-RXN (reverse),DEOXYGUANPHOSPHOR-RXN (reverse),DEOXYRIBOSE-P-ALD-RXN (reverse),DEPHOSPHOCOAKIN-RXN,DETHIOBIOTIN-SYN-RXN,DHBDEHYD-RXN,DHHB-METHYLTRANSFER-RXN,DIAMINOPIMDECARB-RXN,DIAMINOPIMEPIM-RXN,DIHYDRODIPICSYN-RXN,DIHYDROFOLATESYNTH-RXN,DIHYDRONEOPTERIN-MONO-P-DEPHOS-RXN,DIHYDROOROT-RXN (reverse),DIHYDROOROTATE-DEHYDROGENASE-RXN-DI-H-OROTATE/UBIQUINONE-8//OROTATE/CPD-9956.44.,DIHYDROXYISOVALDEHYDRAT-RXN,DIHYDROXYMETVALDEHYDRAT-RXN,DIMETHUROPORDEHYDROG-RXN,DIOHBUTANONEPSYN-RXN,⋯
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,NAD[c],1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,-1,0,⋯
2,NADH[c],-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,⋯
3,PROTON[c],-1,-1,1,0,-1,1,-1,1,1,-1,1,0,-1,1,-1,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,-1,0,0,0,0,0,1,1,0,0,0,2,1,1,0,0,1,0,0,0,1,0,1,1,0,0,-1,0,1,-1,0,3

In [4]:
metabolite_names = stoichiometry[:, "Column1"]
reaction_names = names(stoichiometry)[2:end]

470-element Vector{String}:
 "1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)"
 "1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)"
 "1PFRUCTPHOSN-RXN"
 "1TRANSKETO-RXN (reverse)"
 "2-DEHYDROPANTOATE-REDUCT-RXN (reverse)"
 "2-ISOPROPYLMALATESYN-RXN"
 "2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN"
 "2-OCTAPRENYL-6-OHPHENOL-METHY-RXN"
 "2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN"
 "2-OCTAPRENYLPHENOL-HYDROX-RXN"
 "2.3.1.157-RXN"
 "2.5.1.19-RXN"
 "2.5.1.64-RXN"
 ⋮
 "GLUTDECARBOX-RXN__GLUTDECARBOXB-CPLX"
 "ISOCHORSYN-RXN__ENTC-MONOMER"
 "ISOCHORSYN-RXN__MENF-CPLX"
 "RXN-9535__FABB-CPLX"
 "RXN0-313__CPLX0-201 (reverse)"
 "RXN0-384__H2NEOPTERINP3PYROPHOSPHOHYDRO-MONOMER"
 "RXN0-5186__G7408-MONOMER"
 "SERINE-O-ACETTRAN-RXN__CPLX0-237"
 "SERINE-O-ACETTRAN-RXN__CYSSYNMULTI-CPLX"
 "UNDECAPRENYL-DIPHOSPHATASE-RXN[" ⋯ 57 bytes ⋯ "PROTON.64.__PGPPHOSPHAB-MONOMER"
 "URPHOS-RXN__URPHOS-CPLX"
 "maintenance_reaction"

In [5]:
S = Matrix(stoichiometry[:, 2:end])
B = S .!= 0

489×470 BitMatrix:
 1  1  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  0  1  1  1  1  1  1  1  0  1     1  0  0  1  0  1  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  1  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  1  0  0  1  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1     1  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  1  0  0  1  0  0  0     0  0  0  0  0  1  1  0  0  1  0  1
 0  0  0  0  0  0  1  0  0  1  0  0  0     0 

In [6]:
node_freq = diag(B * Transpose(B))

489-element Vector{Int64}:
  38
  36
 236
   1
  33
  32
   1
  50
  18
   2
   2
 120
   8
   ⋮
   2
   2
   2
   2
   2
   2
   3
   4
   2
   2
   2
   1

In [ ]:
g = wheel_graph(10)
f, ax, p = graphplot(g)
hidedecorations!(ax); hidespines!(ax)
ax.aspect = DataAspect()

display(f)

In [ ]:
using WGLMakie
using NetworkLayout

In [ ]:
WGLMakie.activate!()
set_theme!(resolution=(800, 600))

In [ ]:
g = smallgraph(:dodecahedral)
graphplot(g, layout=Spring(dim=3))